In [ ]:
import tensorrt as trt
from tensorrt_model import TRTModel
from ssd_tensorrt import load_plugins, parse_boxes,TRT_INPUT_NAME, TRT_OUTPUT_NAME
import ctypes
import numpy as np
import cv2
import os
import ctypes
   
mean = 255.0 * np.array([0.5, 0.5, 0.5])
stdev = 255.0 * np.array([0.5, 0.5, 0.5])


def bgr8_to_ssd_input(camera_value):
   x = camera_value
   x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
   x = x.transpose((2, 0, 1)).astype(np.float32)
   x -= mean[:, None, None]
   x /= stdev[:, None, None]
   return x[None, ...]


class ObjectDetector(object):
   
   def __init__(self, engine_path, preprocess_fn=bgr8_to_ssd_input):
       logger = trt.Logger()
       trt.init_libnvinfer_plugins(logger, '')
       load_plugins()
       self.trt_model = TRTModel(engine_path, input_names=[TRT_INPUT_NAME],output_names=[TRT_OUTPUT_NAME, TRT_OUTPUT_NAME + '_1'])
       self.preprocess_fn = preprocess_fn
       
   def execute(self, *inputs):
       trt_outputs = self.trt_model(self.preprocess_fn(*inputs))
       return parse_boxes(trt_outputs)
   
   def __call__(self, *inputs):
       return self.execute(*inputs)


model = ObjectDetector('ssd_mobilenet_v2_coco.engine')

In [ ]:
import ipywidgets.widgets as widgets
from IPython.display import display, HTML
#use traitlets and widgets to display the image in Jupyter Notebook
import traitlets
from traitlets.config.configurable import SingletonConfigurable

#use opencv to covert the depth image to RGB image for displaying purpose
#pip install opencv-contrib-python --upgrade
#Used for tracking the first "label 1" the camera sees
import cv2
import numpy as np

#using realsense to capture the color and depth image
import pyrealsense2 as rs

#multi-threading is used to capture the image in real time performance
import threading

class Camera(SingletonConfigurable):
    
    #this changing of this value will be captured by traitlets
    color_value = traitlets.Any()
    
    def __init__(self):
        super(Camera, self).__init__()
        
        #configure the color and depth sensor
        self.pipeline = rs.pipeline()
        self.configuration = rs.config()  
        
        #set resolution for the color camera
        self.color_width = 640
        self.color_height = 480
        self.color_fps = 30
        self.configuration.enable_stream(rs.stream.color, self.color_width, self.color_height, rs.format.bgr8, self.color_fps)

        #set resolution for the depth camera
        self.depth_width = 640
        self.depth_height = 480
        self.depth_fps = 30
        self.configuration.enable_stream(rs.stream.depth, self.depth_width, self.depth_height, rs.format.z16, self.depth_fps)

        #flag to control the thread
        self.thread_runnning_flag = False
        
        #start the RGBD sensor
        self.pipeline.start(self.configuration)
        self.pipeline_started = True
        frames = self.pipeline.wait_for_frames()

        #start capture the first color image
        color_frame = frames.get_color_frame()   
        image = np.asanyarray(color_frame.get_data())
        self.color_value = image

        #start capture the first depth image
        depth_frame = frames.get_depth_frame()           
        self.depth_image = np.asanyarray(depth_frame.get_data())
        depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(self.depth_image, alpha=0.03), cv2.COLORMAP_JET)
        self.depth_value = depth_colormap   

    def _capture_frames(self):
        while(self.thread_runnning_flag==True): #continue until the thread_runnning_flag is set to be False
            frames = self.pipeline.wait_for_frames() #receive data from RGBD sensor
            
            color_frame = frames.get_color_frame() #get the color image
            image = np.asanyarray(color_frame.get_data()) #convert color image to numpy array
            self.color_value = image #assign the numpy array image to the color_value variable 

            depth_frame = frames.get_depth_frame() #get the depth image           
            self.depth_image = np.asanyarray(depth_frame.get_data()) #convert depth data to numpy array
            #conver depth data to BGR image for displaying purpose
            depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(self.depth_image, alpha=0.03), cv2.COLORMAP_JET)
            self.depth_value = depth_colormap #assign the color BGR image to the depth value
    
    def start(self): #start the data capture thread
        if self.thread_runnning_flag == False: #only process if no thread is running yet
            self.thread_runnning_flag=True #flag to control the operation of the _capture_frames function
            self.thread = threading.Thread(target=self._capture_frames) #link thread with the function
            self.thread.start() #start the thread

    def stop(self): #stop the data capture thread
        if self.thread_runnning_flag == True:
            self.thread_runnning_flag = False #exit the while loop in the _capture_frames
            self.thread.join() #wait the exiting of the thread 
            
    def getDepth(self,x,y):
        if x > 640:
            x = 620
        if x < 0:
            x = 0
        if y > 480:
            y = 460
        if y < 0:
            y = 0
        return self.depth_image[int(x),int(y)]
            

def bgr8_to_jpeg(value):#convert numpy array to jpeg coded data for displaying 
    return bytes(cv2.imencode('.jpg',value)[1])

#create a camera object
camera = Camera.instance()
camera.start() # start capturing the data

In [ ]:
import ipywidgets.widgets as widgets
from IPython.display import display, HTML

width = 640
height = 480

image_widget = widgets.Image(format='jpeg', width=300, height=300)
label_widget = widgets.IntText(value=1, description='tracked label')
display(widgets.VBox([
    widgets.HBox([image_widget,]),
    label_widget
]))

import time
from RobotClass import Robot

#initialize the Robot class
robot = Robot()

# Initialize the tracker
tracker = cv2.TrackerCSRT_create()
tracker_initialized = False

def processing(change):
    global tracker_initialized, tracker

    image = change['new']  # This is your camera frame input

    imgsized= cv2.resize(image,(300,300))
    

    if not tracker_initialized:
        # compute all detected objects
        detections = model(imgsized)

        for det in detections:
            # Assuming 'det' has the properties 'label', 'confidence', 'xmin', 'ymin', 'xmax', 'ymax'
            if det['label'] == 1 and det['confidence'] > 0.5:
                bbox = (det['xmin'], det['ymin'], det['xmax'] - det['xmin'], det['ymax'] - det['ymin'])
                tracker.init(image, bbox)
                tracker_initialized = True
                break

    if tracker_initialized:
        success, bbox = tracker.update(image)  # Update the tracker
        if success:
            x, y, w, h = map(int, bbox)
            center_x = x + w / 2.0
            center_y = y + h / 2.0

            # Determine the midpoint of the frame's width
            frame_midpoint = image.shape[1] // 2
            # Calculate 10% of the frame's width to define the center zone
            center_zone_width = image.shape[1] * 0.1
            left_center_zone = frame_midpoint - (center_zone_width / 2)
            right_center_zone = frame_midpoint + (center_zone_width / 2)

            # Check if the center point is near the left or right edge or has moved off-frame
            if center_x < 10:
                print("Action: Near left edge or moved off-frame to the left")
                robot.left(0.5)  # Turn left
            elif center_x > image.shape[1] - 10:
                print("Action: Near right edge or moved off-frame to the right")
                robot.right(0.5)  # Turn right

            # Check if the center point is within the center zone
            elif left_center_zone <= center_x <= right_center_zone:
                print("Action: In the center zone of the frame")
                robot.forward(0.3)  # Move forward if roughly centered
            elif center_x < frame_midpoint:
                print("Action: In the left half of the frame")
                robot.forwardleft(0.5)  # Turn left
            elif center_x > frame_midpoint:
                print("Action: In the right half of the frame")
                robot.forwardright(0.5)  # Turn right

            # Draw the updated bounding box and center point
            cv2.rectangle(image, (x, y), (x + w, y + h), (255, 0, 0), 2)
            cv2.circle(image, (center_x, center_y), 5, (0, 255, 0), -1)  # Draw center point

    
            

camera.observe(processing, names='color_value')  # Call 'processing' on each new frame